# Exercise 2

## Setup

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist
import seaborn as sns
import math
import statistics

sns.set_theme(context='notebook', style='whitegrid', palette='colorblind', font='serif', font_scale=1.5)
plt.rcParams["font.size"] = 22
plt.rcParams["figure.figsize"] = (6/2.54, 4/2.54)
matplotlib.use("pgf")
plt.style.use('seaborn-v0_8-poster')
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
img_folder = "imgs/"

In [35]:
birds = pd.read_csv('./birds2025.csv', sep=';')
birds.head()

,species,group,length,wspan,weight,AR,wload,back,belly,ftype
0,naurulokki,laridae,34-38,86-99,200-350,8.13,0.31,light grey,white,B
1,harmaalokki,laridae,55-65,123-148,800-1300,8.24,0.64,bluish grey,white,B
2,isolokki,laridae,63-68,138-158,1000-1800,8.24,0.66,bluish grey,white,B
3,kalatiira,sternidae,36-42,70-80,100-145,9.14,0.24,grey,white,B
4,lapintiira,sternidae,33-37,66-77,90-130,8.97,0.20,grey,white,B


## Tasks

### Subtask a

Replace interval columns with mean values.

In [36]:
def meanval(a: str) -> float:
    a_split = a.split('-')
    return (float(a_split[0]) + float(a_split[1])) / 2

In [37]:
birds['mean_length'] = birds['length'].apply(lambda x: meanval(x))
birds['mean_wspan'] = birds['wspan'].apply(lambda x: meanval(x))
birds['mean_weight'] = birds['weight'].apply(lambda x: meanval(x))
birds.drop(columns=['length', 'wspan', 'weight'], inplace=True)
birds.head()

,species,group,AR,wload,back,belly,ftype,mean_length,mean_wspan,mean_weight
0,naurulokki,laridae,8.13,0.31,light grey,white,B,36.0,92.5,275.0
1,harmaalokki,laridae,8.24,0.64,bluish grey,white,B,60.0,135.5,1050.0
2,isolokki,laridae,8.24,0.66,bluish grey,white,B,65.5,148.0,1400.0
3,kalatiira,sternidae,9.14,0.24,grey,white,B,39.0,75.0,122.5
4,lapintiira,sternidae,8.97,0.20,grey,white,B,35.0,71.5,110.0


Compute BMI and WSI

In [38]:
birds['BMI'] = (birds['mean_weight'] / 1000) / (birds['mean_length'] / 100) ** 2
birds.head()

,species,group,AR,wload,back,belly,ftype,mean_length,mean_wspan,mean_weight,BMI
0,naurulokki,laridae,8.13,0.31,light grey,white,B,36.0,92.5,275.0,2.121914
1,harmaalokki,laridae,8.24,0.64,bluish grey,white,B,60.0,135.5,1050.0,2.916667
2,isolokki,laridae,8.24,0.66,bluish grey,white,B,65.5,148.0,1400.0,3.263213
3,kalatiira,sternidae,9.14,0.24,grey,white,B,39.0,75.0,122.5,0.805391
4,lapintiira,sternidae,8.97,0.20,grey,white,B,35.0,71.5,110.0,0.897959


In [39]:
birds['WSI'] = birds['mean_wspan'] / birds['mean_length']
birds.head()

,species,group,AR,wload,back,belly,ftype,mean_length,mean_wspan,mean_weight,BMI,WSI
0,naurulokki,laridae,8.13,0.31,light grey,white,B,36.0,92.5,275.0,2.121914,2.569444
1,harmaalokki,laridae,8.24,0.64,bluish grey,white,B,60.0,135.5,1050.0,2.916667,2.258333
2,isolokki,laridae,8.24,0.66,bluish grey,white,B,65.5,148.0,1400.0,3.263213,2.259542
3,kalatiira,sternidae,9.14,0.24,grey,white,B,39.0,75.0,122.5,0.805391,1.923077
4,lapintiira,sternidae,8.97,0.20,grey,white,B,35.0,71.5,110.0,0.897959,2.042857


Split dataset.

In [40]:
birds.set_index('species', inplace=True)
groups = birds["group"]
birds.drop(columns=['group'], inplace=True)
birds.head()

,AR,wload,back,belly,ftype,mean_length,mean_wspan,mean_weight,BMI,WSI
species,,,,,,,,,,
naurulokki,8.13,0.31,light grey,white,B,36.0,92.5,275.0,2.121914,2.569444
harmaalokki,8.24,0.64,bluish grey,white,B,60.0,135.5,1050.0,2.916667,2.258333
isolokki,8.24,0.66,bluish grey,white,B,65.5,148.0,1400.0,3.263213,2.259542
kalatiira,9.14,0.24,grey,white,B,39.0,75.0,122.5,0.805391,1.923077
lapintiira,8.97,0.20,grey,white,B,35.0,71.5,110.0,0.897959,2.042857


Assess numerical feature distribution.

In [ ]:
for i in birds.select_dtypes(include=['float64']).columns:
    plt.hist(birds[i], bins = 25)
    plt.title(i)
    plt.xlabel(i)
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(img_folder + f'hist_{i}.pdf', bbox_inches='tight')
    plt.show()
    plt.clf()

C:\Users\stefa\AppData\Local\Temp\ipykernel_35112\3058707084.py:8: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


Scale numerical features.

In [42]:
scaler = MinMaxScaler()
num_cols = birds.select_dtypes(include=['float64']).columns
birds[num_cols] = pd.DataFrame(scaler.fit_transform(birds[num_cols]), columns=num_cols, index=birds.index)
birds.head()

,AR,wload,back,belly,ftype,mean_length,mean_wspan,mean_weight,BMI,WSI
species,,,,,,,,,,
naurulokki,0.762605,0.060185,light grey,white,B,0.152727,0.334204,0.022779,0.214481,0.879054
harmaalokki,0.785714,0.212963,bluish grey,white,B,0.327273,0.558747,0.093394,0.343958,0.670654
isolokki,0.785714,0.222222,bluish grey,white,B,0.367273,0.624021,0.125285,0.400416,0.671464
kalatiira,0.974790,0.027778,grey,white,B,0.174545,0.242820,0.008884,0.000000,0.446080
lapintiira,0.939076,0.009259,grey,white,B,0.145455,0.224543,0.007745,0.015081,0.526316


Pairwise distances based on numerical features.

In [ ]:
cdist_matrix = cdist(birds[num_cols], birds[num_cols], metric='euclidean')
sns.heatmap(cdist_matrix)
plt.title('Pairwise Euclidean distances between numerical features of birds')
plt.xlabel('Row number')
plt.ylabel('Row number')
plt.tight_layout()
plt.savefig(img_folder + 'euclidean_distance_matrix.pdf', bbox_inches='tight')
plt.show()
plt.clf()
display(pd.DataFrame(cdist_matrix))

C:\Users\stefa\AppData\Local\Temp\ipykernel_35112\3277567804.py:8: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.000000,0.412240,0.494961,0.537179,0.458502,0.614269,0.844823,0.798031,0.483868,0.602311,...,0.945243,1.045541,0.806295,0.720158,0.606286,0.658066,0.431060,0.557765,0.378187,1.351111
1,0.412240,0.000000,0.100755,0.607504,0.588298,0.713402,0.894036,0.788759,0.624433,0.722079,...,0.742517,0.708791,0.667645,0.790164,0.580133,0.601454,0.489592,0.597813,0.404217,1.111911
2,0.494961,0.100755,0.000000,0.693322,0.678988,0.793716,0.963168,0.843763,0.709864,0.806920,...,0.745752,0.656002,0.676412,0.849024,0.604778,0.625901,0.522463,0.646808,0.425812,1.036811
3,0.537179,0.607504,0.693322,0.000000,0.097289,0.558724,0.747743,0.806091,0.517132,0.479969,...,0.941824,1.122554,0.982712,0.819173,0.906038,0.893173,0.843769,0.772793,0.819415,1.658097
4,0.458502,0.588298,0.678988,0.097289,0.000000,0.527917,0.735403,0.788746,0.468235,0.450379,...,0.953783,1.136505,0.960636,0.781061,0.860565,0.856511,0.783774,0.727560,0.761523,1.634855
5,0.614269,0.713402,0.793716,0.558724,0.527917,0.000000,0.243948,0.302904,0.182761,0.112892,...,0.828375,1.069008,0.626720,0.334024,0.616545,0.584045,0.690634,0.476636,0.792848,1.488891
6,0.844823,0.894036,0.963168,0.747743,0.735403,0.243948,0.000000,0.226925,0.414222,0.317034,...,0.825348,1.103945,0.639994,0.325780,0.701961,0.642132,0.841146,0.566475,0.969449,1.545433
7,0.798031,0.788759,0.843763,0.806091,0.788746,0.302904,0.226925,0.000000,0.412983,0.403661,...,0.741083,0.959467,0.423306,0.253123,0.543506,0.481634,0.720250,0.465289,0.857201,1.340114
8,0.483868,0.624433,0.709864,0.517132,0.468235,0.182761,0.414222,0.412983,0.000000,0.182537,...,0.905566,1.088247,0.646703,0.434527,0.607821,0.603690,0.630586,0.503395,0.708963,1.453638
9,0.602311,0.722079,0.806920,0.479969,0.450379,0.112892,0.317034,0.403661,0.182537,0.000000,...,0.889452,1.125469,0.717948,0.437411,0.699389,0.671926,0.745136,0.554984,0.826393,1.556032


In [ ]:
# Numerical feature correlations
corr = birds[num_cols].corr(method="spearman")
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation matrix of numerical features of birds')
plt.tight_layout()
plt.savefig(img_folder + 'num_correlation_matrix.pdf', bbox_inches='tight')
plt.show()
plt.clf()

C:\Users\stefa\AppData\Local\Temp\ipykernel_35112\1935167451.py:7: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [45]:
birds = birds.drop(columns=["mean_length"])
num_cols = birds.select_dtypes(include=['float64']).columns

### Subtask b

Identify categorical features.

In [46]:
categ_cols = birds.select_dtypes(include=['object']).columns

Defining distance measure for color features.

In [47]:
main_colors = ['black', 'white', 'brown', 'grey', 'beige']

def split_color(a: str) -> list:
    if '-' in a:
        a_split = a.split('-')
    elif ' ' in a:
        a_split = a.split(' ')
    else:
        a_split = [a]
    return a_split

def color_similarity(a: str, b: str) -> float:
    a_split = split_color(a)
    b_split = split_color(b)

    intersection = set(a_split).intersection(set(b_split))
    
    if a_split == b_split: # exact match
        return 1.0
    elif any(i_color in main_colors for i_color in intersection): # same main color
        return 0.5
    else: # no match
        return 0.0


Overlap distance.

In [48]:
def get_categ_dist(row1, row2) -> float:
    sim = 0.0

    for col in categ_cols:
        sim += color_similarity(row1[col], row2[col])

    return 1-(sim / len(categ_cols))


Combined pairwise distance (numerical + categorical).

In [49]:
def l2(row_1, row_2):
    sum_of_squares = 0
    for col in num_cols:
        sum_of_squares += abs(row_1[col] - row_2[col]) ** 2
    dist = math.sqrt(sum_of_squares)
    return dist

def calculate_pairwise_distance(df, distance_function):
    num_rows = len(df)
    distances = [[0.0] * num_rows for _ in range(num_rows)]
    for i in range(num_rows):
        for j in range(num_rows): #Calculating 2 times, just to doublecheck
            row_1 = df.iloc[i]
            row_2 = df.iloc[j]

            dist = distance_function(row_1, row_2)
            distances[i][j] = dist

    return pd.DataFrame(distances)

def create_distance_list(distances):
    distances_list = []
    for i in range(len(distances)):
        for j in range(i + 1, len(distances)):
            distances_list.append(distances.iloc[i, j])
    return distances_list

def get_std_dev(distance_list):
    std_dev = statistics.stdev(distance_list)
    print(f"Standard deviation of the distances: {std_dev}")
    return std_dev

In [50]:
# Numerical features
pairwise_distances = calculate_pairwise_distance(birds, l2)

l2_distance_list = create_distance_list(pairwise_distances)

l2_std = get_std_dev(l2_distance_list)

Standard deviation of the distances: 0.31935723783892983


In [51]:
# Categorical features
categ_pairwise = calculate_pairwise_distance(birds, get_categ_dist)

categ_std = get_std_dev(create_distance_list(categ_pairwise))


Standard deviation of the distances: 0.2534869388690408


In [52]:
def mixed_metric(row1, row2):
    lambda_ = 0.8
    return lambda_*l2(row1, row2)/l2_std+(1-lambda_)*get_categ_dist(row1, row2)/categ_std

In [53]:
mixed_distances = calculate_pairwise_distance(birds, mixed_metric)
mixed_distances.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.000000,1.067040,1.248858,1.476038,1.279917,2.052823,2.634448,2.788005,1.712886,2.014624,...,2.241334,2.798208,2.510961,2.461423,2.148025,2.267237,1.842521,2.036032,1.684309,4.032915
1,1.067040,0.000000,0.231652,1.604440,1.533056,2.198915,2.678221,2.711650,1.933140,2.200846,...,2.010315,2.076380,2.197252,2.583768,2.103421,2.161926,1.998955,2.140307,1.793508,3.523801
2,1.248858,0.231652,0.000000,1.799843,1.739062,2.375921,2.828945,2.828302,2.122370,2.389379,...,2.052152,1.980289,2.212477,2.710457,2.152387,2.214777,2.062792,2.247895,1.831073,3.350709
3,1.476038,1.604440,1.799843,0.000000,0.232563,1.903838,2.384083,2.806963,1.786068,1.690151,...,2.390302,3.045942,2.835490,2.708256,2.914759,2.874636,2.894798,2.585616,2.825748,4.841463
4,1.279917,1.533056,1.739062,0.232563,0.000000,1.837072,2.360893,2.764829,1.676622,1.630988,...,2.387537,3.048038,2.768830,2.614079,2.790876,2.770329,2.735737,2.463386,2.668237,4.764438


In [ ]:
# Mutual information between categorical features
from sklearn.metrics import mutual_info_score

categ_data = birds[categ_cols].astype('category').apply(lambda x: x.cat.codes)
corr = categ_data.corr(method=mutual_info_score)
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Mutual information matrix of categorical features of birds')
plt.tight_layout()
plt.savefig(img_folder + 'categ_correlation_matrix.pdf', bbox_inches='tight')
plt.show()
plt.clf()

C:\Users\stefa\AppData\Local\Temp\ipykernel_35112\103830748.py:10: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [55]:
birds = birds.drop(columns=["back"])
categ_cols = birds.select_dtypes(include=['object']).columns

### Subtask c

In [56]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from sklearn.metrics import normalized_mutual_info_score, adjusted_mutual_info_score, rand_score, adjusted_rand_score

In [57]:
mixed_distances = calculate_pairwise_distance(birds, mixed_metric)
mixed_distances.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.000000,0.935541,1.117359,1.344539,1.148418,1.921323,2.502949,2.788005,1.581387,1.883125,...,2.307084,2.666709,2.576710,2.395673,2.082275,2.201488,1.842521,1.970282,1.684309,4.032915
1,0.935541,0.000000,0.231652,1.472940,1.401557,2.067416,2.546722,2.711650,1.801641,2.069347,...,1.944565,1.944881,2.263002,2.518018,2.037671,2.096177,1.998955,2.074558,1.793508,3.523801
2,1.117359,0.231652,0.000000,1.668344,1.607563,2.244421,2.697446,2.828302,1.990871,2.257880,...,1.986402,1.848790,2.278227,2.644708,2.086637,2.149028,2.062792,2.182146,1.831073,3.350709
3,1.344539,1.472940,1.668344,0.000000,0.232563,1.772339,2.252584,2.806963,1.654569,1.558651,...,2.324552,2.914443,3.032738,2.642507,2.849010,2.808887,2.894798,2.519866,2.825748,4.841463
4,1.148418,1.401557,1.607563,0.232563,0.000000,1.705572,2.229394,2.764829,1.545123,1.499488,...,2.321787,2.916539,2.966079,2.548329,2.725126,2.704580,2.735737,2.397636,2.668237,4.764438


In [58]:
bird_map = {
    "charadriiformes": "1",
    "lari": "1.1",
    "laridae": "1.1.1",
    "sternidae": "1.1.2",
    "charadrii": "1.2",
    "scolopacidae": "1.2.1",
    "charadriidae": "1.2.2",
    "haematopodidae": "1.2.3",
    "anatidae": "2",
    "anatinae": "2.1",
    "dabbling ducks": "2.1.1",
    "diving ducks": "2.1.2",
    "anserinae": "2.2",
    "gruifores": "3",
    "gruidae": "3.1",
    "rallidae": "3.2",
    "phalacrocoracidae": "4",
    "podicipedidae": "5",
    "gaviidae": "6",
    "ardeidae": "7",
    "accipitriformes": "8",
    "accipitridae": "8.1"
}

In [59]:
birds['group'] = groups
birds['group_ID'] = birds['group'].map(bird_map)
birds

,AR,wload,belly,ftype,mean_wspan,mean_weight,BMI,WSI,group,group_ID
species,,,,,,,,,,
naurulokki,0.762605,0.060185,white,B,0.334204,0.022779,0.214481,0.879054,laridae,1.1.1
harmaalokki,0.785714,0.212963,white,B,0.558747,0.093394,0.343958,0.670654,laridae,1.1.1
isolokki,0.785714,0.222222,white,B,0.624021,0.125285,0.400416,0.671464,laridae,1.1.1
kalatiira,0.974790,0.027778,white,B,0.242820,0.008884,0.000000,0.446080,sternidae,1.1.2
lapintiira,0.939076,0.009259,white,B,0.224543,0.007745,0.015081,0.526316,sternidae,1.1.2
suokukko,0.468487,0.083333,white,C,0.099217,0.007745,0.144386,0.405667,scolopacidae,1.2.1
taivaanvuohi,0.296218,0.101852,white,C,0.070496,0.006834,0.109788,0.239971,scolopacidae,1.2.1
lehtokurppa,0.247899,0.166667,dappled beige,C,0.164491,0.025513,0.274415,0.306220,scolopacidae,1.2.1
karikukko,0.569328,0.125000,white,C,0.091384,0.007745,0.222778,0.527379,scolopacidae,1.2.1


In [ ]:
def plot_dendrogram(model: AgglomerativeClustering, n_cluster, linkage, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)

    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]).astype(float)
    threshold = (linkage_matrix[-(n_cluster-1), 2] + linkage_matrix[-n_cluster, 2]) / 2
    dendrogram(linkage_matrix, color_threshold=threshold,  above_threshold_color='black', leaf_font_size=14, **kwargs)
    plt.axhline(y=threshold, color='r', linestyle='--')
    plt.title(f'Hierarchical clustering dendrogram\n(k={n_cluster}, linkage={linkage})')
    plt.xlabel('Sample biological group')
    plt.ylabel('Distance')
    plt.tight_layout()
    plt.savefig(img_folder + f'dendrogram_k{n_cluster}_linkage{linkage}.pdf', bbox_inches='tight')
    plt.show()
    plt.clf()


In [71]:
def hier_clust(data, n_clusters, method):
    model = AgglomerativeClustering(n_clusters=n_clusters, linkage=method, compute_distances=True, metric='precomputed')
    #model.fit(data)
    
    labels = model.fit_predict(data)
    plot_dendrogram(model, n_cluster=n_clusters, linkage=method, labels=birds['group_ID'], leaf_rotation=90)
    return labels

In [72]:
for k in range(5,16):
    for linkage_method in ['single', 'complete', 'average']:
        labels = hier_clust(mixed_distances, k, linkage_method)
        print(labels)
        print("Singletons: ",(np.unique(labels, return_counts=True)[1] == 1).sum())
        print(f"k={k}, method={linkage_method}")
        print(f"Normalized Mutual Information Score: {normalized_mutual_info_score(groups, labels, average_method='geometric')}")
        print(f"Adjusted Mutual Information Score: {adjusted_mutual_info_score(groups, labels, average_method='geometric')}")
        print(f"Rand Index: {rand_score(groups, labels)}")
        print(f"Adjusted Rand Index: {adjusted_rand_score(groups, labels)}")

c:\Users\stefa\OneDrive\Documents\uni\master's\EIT\Aalto\methods_of_data_mining\groupwork\.venv\Lib\site-packages\scipy\cluster\hierarchy.py:3542: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ivl.append(labels[int(i - n)])
C:\Users\stefa\AppData\Local\Temp\ipykernel_35112\3161648863.py:24: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
c:\Users\stefa\OneDrive\Documents\uni\master's\EIT\Aalto\methods_of_data_mining\groupwork\.venv\Lib\site-packages\scipy\cluster\hierarchy.py:3542: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ivl.append(labels[int(i - n)])


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 4 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 0 0 0 0 2]
Singletons:  3
k=5, method=single
Normalized Mutual Information Score: 0.26563459322298494
Adjusted Mutual Information Score: 0.054354980407690226
Rand Index: 0.24816326530612245
Adjusted Rand Index: 0.01014573750053738


C:\Users\stefa\AppData\Local\Temp\ipykernel_35112\3161648863.py:24: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
c:\Users\stefa\OneDrive\Documents\uni\master's\EIT\Aalto\methods_of_data_mining\groupwork\.venv\Lib\site-packages\scipy\cluster\hierarchy.py:3542: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ivl.append(labels[int(i - n)])


[1 1 1 1 1 4 4 4 4 4 4 4 4 4 4 1 4 2 4 4 4 2 2 2 0 0 0 4 4 4 2 4 0 2 2 2 4
 2 2 4 4 0 4 4 4 4 1 4 1 3]
Singletons:  1
k=5, method=complete
Normalized Mutual Information Score: 0.4957513582474408
Adjusted Mutual Information Score: 0.3119081164375431
Rand Index: 0.689795918367347
Adjusted Rand Index: 0.13777455073516065


C:\Users\stefa\AppData\Local\Temp\ipykernel_35112\3161648863.py:24: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
c:\Users\stefa\OneDrive\Documents\uni\master's\EIT\Aalto\methods_of_data_mining\groupwork\.venv\Lib\site-packages\scipy\cluster\hierarchy.py:3542: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ivl.append(labels[int(i - n)])


...

C:\Users\stefa\AppData\Local\Temp\ipykernel_35112\3161648863.py:24: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
c:\Users\stefa\OneDrive\Documents\uni\master's\EIT\Aalto\methods_of_data_mining\groupwork\.venv\Lib\site-packages\scipy\cluster\hierarchy.py:3542: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ivl.append(labels[int(i - n)])


[ 2  2  2  5  5  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0
 13  7  8  0  0  0  9  0 10  0  0  0  0 14 12  4  3  1  0  0  0  0  0  0
  0  6]
Singletons:  12
k=15, method=single
Normalized Mutual Information Score: 0.640100118511686
Adjusted Mutual Information Score: 0.37973193956832246
Rand Index: 0.6146938775510205
Adjusted Rand Index: 0.13375422111290558


C:\Users\stefa\AppData\Local\Temp\ipykernel_35112\3161648863.py:24: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
c:\Users\stefa\OneDrive\Documents\uni\master's\EIT\Aalto\methods_of_data_mining\groupwork\.venv\Lib\site-packages\scipy\cluster\hierarchy.py:3542: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ivl.append(labels[int(i - n)])


[ 2  2  2  2  2 10  6  6 10 10 10 10 10 10 10 12  0  4 14 14 14  4  4  4
  1 11  1  6  6  6  3  6  1  4  3  4 14  8  8  5  5 13  0  0  0  0  9  0
  9  7]
Singletons:  4
k=15, method=complete
Normalized Mutual Information Score: 0.7691437610002476
Adjusted Mutual Information Score: 0.5401761396619436
Rand Index: 0.916734693877551
Adjusted Rand Index: 0.42901925660546353
[ 0  0  0  0  0  6  6  4  6  6  6  6  6  6  6 13  3  2  4  4  4  2  2  2
  8  7  8  4  4  4  9  4 14  2  2  2  4 10 10  1  1 11  3  3  3  3  5  3
  5 12]
Singletons:  6
k=15, method=average
Normalized Mutual Information Score: 0.7658274963912931
Adjusted Mutual Information Score: 0.5507725790602598
Rand Index: 0.9069387755102041
Adjusted Rand Index: 0.43136472466081405


C:\Users\stefa\AppData\Local\Temp\ipykernel_35112\3161648863.py:24: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


### Subtask d

In [63]:
final_labels = AgglomerativeClustering(n_clusters=11, linkage='average', compute_distances=True, metric='precomputed').fit_predict(mixed_distances)
birds['predicted_group'] = final_labels
birds['group'] = groups
birds

,AR,wload,belly,ftype,mean_wspan,mean_weight,BMI,WSI,group,group_ID,predicted_group
species,,,,,,,,,,,
naurulokki,0.762605,0.060185,white,B,0.334204,0.022779,0.214481,0.879054,laridae,1.1.1,4
harmaalokki,0.785714,0.212963,white,B,0.558747,0.093394,0.343958,0.670654,laridae,1.1.1,4
isolokki,0.785714,0.222222,white,B,0.624021,0.125285,0.400416,0.671464,laridae,1.1.1,4
kalatiira,0.974790,0.027778,white,B,0.242820,0.008884,0.000000,0.446080,sternidae,1.1.2,4
lapintiira,0.939076,0.009259,white,B,0.224543,0.007745,0.015081,0.526316,sternidae,1.1.2,4
suokukko,0.468487,0.083333,white,C,0.099217,0.007745,0.144386,0.405667,scolopacidae,1.2.1,3
taivaanvuohi,0.296218,0.101852,white,C,0.070496,0.006834,0.109788,0.239971,scolopacidae,1.2.1,3
lehtokurppa,0.247899,0.166667,dappled beige,C,0.164491,0.025513,0.274415,0.306220,scolopacidae,1.2.1,3
karikukko,0.569328,0.125000,white,C,0.091384,0.007745,0.222778,0.527379,scolopacidae,1.2.1,3


Describe clusters by numerical features.

In [64]:
birds[list(num_cols)+["predicted_group"]].groupby('predicted_group').agg(['mean', 'std'])

AR               wload           mean_wspan            \
                     mean       std      mean       std       mean       std   
predicted_group                                                                
0                0.410714  0.026155  0.376157  0.044627   0.756527  0.168515   
1                0.277311  0.132146  0.084491  0.063225   0.417755  0.138357   
2                0.600540  0.097241  0.567460  0.123472   0.216710  0.052523   
3                0.379552  0.180598  0.144290  0.116249   0.092979  0.067750   
4                0.849580  0.099259  0.106481  0.103107   0.396867  0.183818   
5                0.142857       NaN  0.231481       NaN   0.973890       NaN   
6                0.615546       NaN  0.125000       NaN   0.167102       NaN   
7                0.670168       NaN  0.912037       NaN   1.000000       NaN   
8                0.258403  0.044566  0.148148  0.065473   0.582245  0.184623   
9                0.254202       NaN  0.453704       NaN   0.242820       NaN   
10               0.966387  0.047537  0.837963  0.229155   0.403394  0.038771   

                mean_weight                 BMI                 WSI            
                       mean       std      mean       std      mean       std  
predicted_group                                                                
0                  0.320046  0.089845  0.447710  0.113120  0.320153  0.146938  
1                  0.055752  0.030804  0.285905  0.104731  0.673824  0.179473  
2                  0.065799  0.014064  0.476174  0.108861  0.202101  0.112215  
3                  0.016545  0.023030  0.206823  0.109524  0.336488  0.129346  
4                  0.051617  0.054210  0.194787  0.183820  0.638713  0.165527  
5                  0.498861       NaN  1.000000       NaN  0.776087       NaN  
6                  0.007973       NaN  0.247466       NaN  1.000000       NaN  
7                  1.000000       NaN  0.639363       NaN  0.124245       NaN  
8                  0.106606  0.019973  0.158155  0.143726  0.253169  0.096394  
9                  0.066059       NaN  0.761311       NaN  0.515712       NaN  
10                 0.186788  0.054765  0.704662  0.160457  0.277291  0.022317

In [65]:
num_data = birds[list(num_cols) + ["predicted_group"]].copy()
df = pd.melt(num_data, num_data.columns[-1], num_data.columns[:-1])

g = sns.FacetGrid(
    df,
    col="variable",
    hue="predicted_group",
    col_wrap=3,
    height=8/2.54,
    sharex=False,
    sharey=False,
    legend_out=True
)
g.map(sns.kdeplot, "value", fill=True, warn_singular=False)
g.add_legend(title="Cluster")  
# plt.tight_layout()
plt.savefig(img_folder + 'kde_plots.pdf', bbox_inches='tight')
plt.clf()
plt.show()

C:\Users\stefa\AppData\Local\Temp\ipykernel_35112\1045383058.py:19: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


Describe clusters by categorical features.

In [66]:
for col in categ_cols:
    print(f"\nFeature: {col}")
    display(pd.crosstab(birds['predicted_group'], birds[col], normalize='index') * 100)


Feature: belly


belly,beige,black,black-white,brown,brown-white,dappled beige,dappled brown,dark brown,dark grey,grey,greyish black,greyish white,light grey,reddish brown,white
predicted_group,,,,,,,,,,,,,,,
0,0.000000,25.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,25.000000,0.0,0.0,25.0,0.000000,25.000000
1,0.000000,0.0,25.000000,12.5,37.5,0.000000,0.000000,12.500000,0.000000,0.000000,0.0,0.0,0.0,0.000000,12.500000
2,14.285714,0.0,0.000000,0.0,0.0,0.000000,0.000000,28.571429,0.000000,0.000000,0.0,0.0,0.0,28.571429,28.571429
3,27.777778,0.0,5.555556,0.0,0.0,5.555556,5.555556,0.000000,5.555556,5.555556,0.0,0.0,0.0,0.000000,44.444444
4,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,100.000000
5,0.000000,0.0,0.000000,100.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
6,0.000000,100.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
7,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,100.000000
8,0.000000,0.0,0.000000,0.0,0.0,0.000000,50.000000,0.000000,0.000000,0.000000,0.0,50.0,0.0,0.000000,0.000000



Feature: ftype


ftype,A,B,C
predicted_group,,,
0,25.0,0.0,75.0
1,87.5,0.0,12.5
2,0.0,0.0,100.0
3,0.0,0.0,100.0
4,0.0,100.0,0.0
5,100.0,0.0,0.0
6,0.0,0.0,100.0
7,0.0,0.0,100.0
8,0.0,100.0,0.0
